In [192]:
from pybedtools import BedTool
import sys
import pandas as pd

sys.argv=['test.opy','../../universal_data/rmsk/rmsk_GRCh38.txt','../../universal_data/ref/GRCh38/genes.bed']
rmsk=sys.argv[1]
gene_anno=sys.argv[2]

rmsk_df=pd.read_csv(rmsk,sep='\t')
rmsk_df['index']=rmsk_df[['repClass','repFamily','repName']].agg(':'.join,axis=1)

rmsk_bed=BedTool.from_dataframe(rmsk_df[['genoName','genoStart','genoEnd','index','swScore','strand']])
conn=rmsk_bed.intersect(anno_bed,s=True,wa=True,wb=True).to_dataframe()
conn[['ensg','gn']]=[x.split(':') for x in conn['blockCount']]

print(len(conn))
conn['class']=[x.split(':')[0] for x in conn['name']]
conn=conn.loc[conn['class'].isin(['SINE','LTR','LINE'])]
conn=conn[['name','gn']].drop_duplicates()
conn[['name','gn']].to_csv('../www/shiny-app/gene_te_net/network.txt',header=True,index=False)



In [193]:
print(len(conn))
conn['class']=[x.split(':')[0] for x in conn['name']]
conn=conn.loc[conn['class'].isin(['SINE','LTR','LINE'])]
conn=conn[['name','gn']].drop_duplicates()
conn[['name','gn']].to_csv('../www/shiny-app/gene_te_net/network.txt',header=True,index=False)


1690201
1218487


,chrom,start,end,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStarts,ensg,gn,class
0,chr1,8388315,8388618,SINE:Alu:AluY,2582,-,chr1,8352397,8848921,ENSG00000142599:RERE,1000,-,ENSG00000142599,RERE,SINE
1,chr1,33554185,33554483,SINE:Alu:AluSc,2285,-,chr1,33513998,34165842,ENSG00000121904:CSMD2,1000,-,ENSG00000121904,CSMD2,SINE
2,chr1,41942894,41943205,SINE:Alu:AluY,2451,-,chr1,41506365,42035925,ENSG00000127124:HIVEP3,1000,-,ENSG00000127124,HIVEP3,SINE
3,chr1,58719764,58720546,LINE:L2:L2a,1393,+,chr1,58715609,58771295,ENSG00000283445:ENSG00000283445,1000,+,ENSG00000283445,ENSG00000283445,LINE
4,chr1,150994812,150995102,SINE:Alu:AluSg,2212,+,chr1,150982249,150995634,ENSG00000143412:ANXA9,1000,+,ENSG00000143412,ANXA9,SINE


In [199]:
conn[['name','gn']].to_csv('../www/shiny-app/gene_te_net/network.txt',header=True,index=False)


In [198]:
len(conn)

558665